In [ ]:
!pip install sklearn
!pip install quantulum3
!pip install transformers
!pip install nltk

     |████████████████████████████████| 10.8MB 2.6MB/s 
     |████████████████████████████████| 102kB 10.0MB/s 
     |████████████████████████████████| 1.1MB 2.8MB/s 
     |████████████████████████████████| 890kB 10.6MB/s 
     |████████████████████████████████| 1.1MB 20.1MB/s 
     |████████████████████████████████| 3.0MB 33.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=8b914bee0c17e8dd660f1bee7b47db5ebc3829f25e1879219d33388ec94f7025
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import os
import re
import io
import pandas as pd
import ast 
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel, BertForPreTraining
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from torchtext import vocab
from torchtext import data    
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader

import nltk
from sklearn.model_selection import train_test_split
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive/MeasEval-main/data/train/train/tsv

/content/drive/My Drive/MeasEval-main/data/train/train/tsv


In [ ]:
files=os.listdir()

In [ ]:
cd ..

/content/drive/My Drive/MeasEval-main/data/train/train


In [ ]:
train=[]
for i in files:
  
  pathtsv=os.path.join('tsv',i[:-3]+'tsv')
  if(not(os.path.exists(pathtsv))):
    continue
  
  filetsv=pd.read_csv(pathtsv,sep = '\t')
  
  entity=filetsv[['annotType','text','other']]
  quant=entity[entity['annotType']=='Quantity']
  quant=quant[['text','other']]
  quant = quant.to_numpy().tolist()
  train.append(quant)
  
  
  
  

In [ ]:
X=[]
label=[]
for i in train:
  for j in i:
    temp=[]
  
    X.append(j[0])
    
    
    
    
    other=j[1]
    
    if(not pd.isnull(other)):
      dictio=ast.literal_eval(other) 
    
      
      if 'unit' in dictio:
        t=j[0].find(dictio['unit'])
        
        t=[m.start() for m in re.finditer(dictio['unit'],j[0])]
        sz=len(dictio['unit'])
        span=[]
        for k in t:
          span.append((k,k+sz))
        
        
        label.append(span)
        

      else:
        label.append([])
    else:
      label.append([])
  
    

In [ ]:
X_train=[]
y_train=[]
for i in range(len(X)):

  character=list(X[i]);
  X_train.append(character)
  l=np.zeros(64)
  
  for k in label[i]:
    l[k[0]:k[1]]=1;
  y_train.append(l)
  

In [ ]:
X_train,X_val,y_train,y_val=train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [ ]:

TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)
TEXT.build_vocab(X_train, max_size=1000)


In [ ]:
vocab_size=len(TEXT.vocab)

In [ ]:
X_train=TEXT.process(X_train)[0]
X_val=TEXT.process(X_val)[0]

X_train[0]

tensor([45,  4,  2, 18, 18,  5,  2, 33, 35,  2,  5,  9, 17, 17,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1])

In [ ]:
row=X_train.shape[1]
X_train=F.pad(input=X_train, pad=(0, 64-row, 0, 0), mode='constant', value=1)

row=X_val.shape[1]
X_val=F.pad(input=X_val, pad=(0, 64-row, 0, 0), mode='constant', value=1)


In [ ]:
train_data = TensorDataset(X_train, torch.from_numpy(np.asarray(y_train)))
val_data = TensorDataset(X_val,torch.from_numpy(np.array(y_val)))

In [ ]:
batch_size = 38
train_loader = DataLoader(train_data, shuffle=True, batch_size = batch_size)
val_loader = DataLoader(val_data, shuffle=True, batch_size = batch_size)

In [ ]:
lr = 1e-4

In [ ]:
class bilstm(nn.Module):
    def __init__(self, vocab_size, output_size, embedding_dim, hidden_dim, n_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.output_size = output_size
        
        self.embedding = nn.Embedding(vocab_size,embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, n_layers, 
                         batch_first=True, bidirectional=True)
        
        self.dropout = nn.Dropout(0.5)
        self.fc = nn.Linear(hidden_dim*2,output_size)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self,x):
        embeds = self.embedding(x)
        lstm, (hn,cn)= self.lstm(embeds)
        
        output = self.dropout(lstm)
        output = self.fc(output)
        output = self.sigmoid(output) 
        return output

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  
model = bilstm(vocab_size, 1, 100, 32, 2)
model = model.to(device)

In [ ]:
print(model)

bilstm(
  (embedding): Embedding(84, 100)
  (lstm): LSTM(100, 32, num_layers=2, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [ ]:
from transformers import AdamW

criterion = nn.BCELoss()
optimizer = AdamW(model.parameters(), lr = 1e-3)

In [ ]:
epochs = 50
for e in range(epochs):
  
  model.train()
  i=0
  train_loss=0
  for seq, y in train_loader:
    model.zero_grad()
    
    y_pred = model(seq.to(device))
    # print(y_pred.shape)
    loss = criterion(y_pred,y.to(device).float())
    loss.backward()
    optimizer.step()
    train_loss += loss.item()*batch_size
    if(i%5==0):
      print("Epoch-{}/{} Iterations-{} loss-{}".format(e+1,epochs,i+1,loss.item()))
    i+=1
  
  
  model.eval()
  val_loss=0
  for seq, y in val_loader:
    model.zero_grad()
    y_pred = model(seq.to(device))
    loss = criterion(y_pred,y.to(device).float())
    val_loss += loss.item()*batch_size
    
    i+=1
  # if(e%5==0):
  #   torch.save(bert_model,'/content/drive/My Drive/Model/bert_{}.pt'.format(e))
  
  print("Epoch-{}/{} train_loss-{} Val_loss-{}".format(e+1,epochs,train_loss/len(train_loader),val_loss/len(val_loader)))


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([38, 64])) that is different to the input size (torch.Size([38, 64, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch-1/50 Iterations-1 loss-0.739799976348877
Epoch-1/50 Iterations-6 loss-0.5517277121543884
Epoch-1/50 Iterations-11 loss-0.38756266236305237
Epoch-1/50 Iterations-16 loss-0.2553079128265381


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([3, 64])) that is different to the input size (torch.Size([3, 64, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([30, 64])) that is different to the input size (torch.Size([30, 64, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)


Epoch-1/50 train_loss-16.142315897345544 Val_loss-6.82407408952713
Epoch-2/50 Iterations-1 loss-0.17642216384410858
Epoch-2/50 Iterations-6 loss-0.13118363916873932
Epoch-2/50 Iterations-11 loss-0.1193750873208046
Epoch-2/50 Iterations-16 loss-0.11138272285461426
Epoch-2/50 train_loss-5.056160780787468 Val_loss-3.9523334860801698
Epoch-3/50 Iterations-1 loss-0.09558525681495667
Epoch-3/50 Iterations-6 loss-0.10795371234416962
Epoch-3/50 Iterations-11 loss-0.10170970112085342
Epoch-3/50 Iterations-16 loss-0.06840620189905167
Epoch-3/50 train_loss-3.5653999626636503 Val_loss-3.0697657436132433
Epoch-4/50 Iterations-1 loss-0.0842113122344017
Epoch-4/50 Iterations-6 loss-0.08208608627319336
Epoch-4/50 Iterations-11 loss-0.0691002607345581
Epoch-4/50 Iterations-16 loss-0.06082301214337349
Epoch-4/50 train_loss-3.058369719609618 Val_loss-2.5719030991196634
Epoch-5/50 Iterations-1 loss-0.059190887957811356
Epoch-5/50 Iterations-6 loss-0.055361777544021606
Epoch-5/50 Iterations-11 loss-0.06801

In [ ]:
pos=0
neg=0
units=[]
for seq, y in val_loader:
    model.zero_grad()
    model.eval()
    y_pred = model(seq.to(device))
    np_out = y_pred.cpu().data.numpy()
    np_act = y.cpu().data.numpy()
    
    np_out=np_out>=0.07

    
    
    
    
    
    for i in range(np_out.shape[0]):
     for j in range(np_out.shape[1]):
       if (np_out[i,j])==1 or (np_act[i,j])==1:
           if (np_out[i,j])==1 and (np_act[i,j])==1:
             pos = pos + 1
           else:
             neg = neg + 1

print(pos/(pos+neg))


0.8313253012048193


In [ ]:
model.eval()
  
for i in range(len(X_val)):
  answer=[]
  t=''
  for j in X_val[i]:
    t=t+j
  answer.append(t)
  temp=[]
  temp.append(X_val[i])
  seq=TEXT.process(temp)[0]
  row=seq.shape[1]
  seq=F.pad(input=seq, pad=(0, 64-row, 0, 0), mode='constant', value=1)
  
  
  y_pred = model(seq.to(device))
  np_out = y_pred.cpu().data.numpy()
  
  
    
  
  
  for i in np_out:
      
      a1=i>=0.07
      a1_rshifted = np.roll(a1, 1)

      starts = a1 & ~a1_rshifted
      ends = ~a1 & a1_rshifted
      starts=np.nonzero(starts)[0]
      ends=np.nonzero(ends)[0]

      for j in range(len(starts)):
        answer.append(t[starts[j]:ends[j]])


  print(answer)
      
  
      

['6–8%', '%']
['≥11.1 mmol/L', 'mmol/L']
['±70 m/s', 'm/s']
['488 ± 14 K', 'K']
['0.13']
['ηnet = 0.5']
['∼0.5–5 Å/s', 'Å/s']
['4–7 kV', 'kV']
['6 kg', 'kg']
['up to 100%', '%']
['two']
['13 kg', 'kg']
['1.85–2.48']
['15']
['1.75']
['5 g', 'g']
['range 1013 cm−3 to 1016 cm−3', 'cm−3', 'cm−3']
['1.5 mm', 'mm']
['5.65 s', 's']
['approximately 54%', '%']
['2.5–10 μm', 'μm']
['over 3 byr', 'byr']
['6967']
['0.06–0.07 mBar', 'mBar']
['three metre', 'met']
['12.1%', '%']
['23.8%', '%']
['n=104']
['95%', '%']
['0.03']
['274 elderly participants', 'ly participants']
['35%', '%']
['1.5‰', '‰']
['from 211 K to 284 K', 'K', 'K']
['< 0.001']
['1000']
['16 times', 'times']
['1 (w/w)']
['range of 34–38 °C', '°C']
['7.5–80']
['∼33.7 Ma', 'Ma']
['84%', '%']
['95%', '%']
['above thirty percent', 'percent']
['2619.60, 2614.73, and 2614.71 m', 'm']
['10,308 employees', 'employees']
['before 6 months', 'months']
['about 40%', '%']
['18']
['0.51–0.85 m.', 'm']
['< .001']
['zero']
['over the 9-year', 'h', '